In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Definición de parámetros
lambda_ = 532e-9  # Longitud de onda en metros (532 nm)
d = 0  # Grosor de la muestra en metros

# Campos eléctricos (en forma de vectores complejos)
Er = np.array([1, 1])  # Campo eléctrico de referencia
Eis = np.array([1, np.exp(1j * np.pi / 8)])  # Campo eléctrico incidente

# Cálculo de la birrefringencia y retardo de fase
Delta_phi = np.pi

# Matriz de Jones del objeto
S = np.array([[np.exp(1j * Delta_phi / 2), 0],
              [0, np.exp(-1j * Delta_phi / 2)]])

# Cálculo del campo eléctrico transmitido
Es = S @ Eis

# Cálculo del campo de salida
Eout = Er + Es

# Cálculo de intensidad detectada
I = np.abs(Eout[0])**2 + np.abs(Eout[1])**2

# Definición de variables para el patrón de interferencia
k = 2 * np.pi / lambda_  # Número de onda
x = np.linspace(-k, k, 500)  # Coordenada espacial
mu = 0  # Desfase global
vartheta = np.arctan2(-np.abs(Er[1]) * np.abs(Es[1]) * np.sin(Delta_phi),
                      np.abs(Er[0]) * np.abs(Es[0]) + np.abs(Er[1]) * np.abs(Es[1]) * np.cos(Delta_phi))

# Cálculo del patrón de interferencia
I_pattern = I + 2 * np.abs(Er[0] * Es[0] + Er[1] * Es[1]) * np.cos(mu + k * x - vartheta)

# Normalización de la intensidad para escalar entre 0 y 255
I_pattern_norm = (I_pattern - I_pattern.min()) / (I_pattern.max() - I_pattern.min()) * 255
I_pattern_norm = np.uint8(I_pattern_norm)

# Convertir a imagen en escala de grises con OpenCV
height = 500  # Altura de la imagen
pattern_image = np.tile(I_pattern_norm, (height, 1))

# Aplicar filtro para resaltar franjas
pattern_image = cv2.GaussianBlur(pattern_image, (5, 5), 0)

# Mostrar el patrón de interferencia como imagen
plt.figure(figsize=(10, 5))
plt.imshow(pattern_image, cmap='gray', aspect='auto')
plt.axis('off')
plt.title("Patrón de Interferencia")
plt.show()


ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Impresión de resultados
print("Campo eléctrico de referencia Er:", Er)
print("Campo eléctrico incidente Eis:", Eis)
print("Matriz de Jones S:")
print(S)
print("Campo eléctrico después de la muestra Es:", Es)
